In [1]:
import pennylane as qml
from pennylane import numpy as np
from braket.aws import AwsDevice

from typing import List

In [2]:
n_wires = 2

In [3]:
local_simulator = qml.device(
    "braket.local.qubit", 
    wires=n_wires,
    shots=1024,
)
local_simulator

<BraketLocalQubitDevice device (wires=2, shots=1024) at 0x7f99b3b9a650>

In [4]:
def circuit(params):
    qml.templates.BasicEntanglerLayers(params, wires=range(n_wires))
    return qml.expval(qml.PauliX(0))

In [5]:
simulated_circuit = qml.QNode(circuit, local_simulator)

In [6]:
def variational_classifier_linear(params):
    meas = simulated_circuit(params)
    rescaled = (meas + 1) / 2
    return rescaled

In [7]:
def squareloss(predictions: List[float], labels: List[int]) -> float:
    loss = 0
    for p, lab in zip(predictions, labels):
        loss += (p - lab)**2  
    return loss / len(labels)

In [8]:
def squareloss_cost(params, label: int) -> float:
    prediction = variational_classifier_linear(params)
    return squareloss([prediction], [label])

In [9]:
n_variational_layers = 1
np.random.seed(2165)
initial_params = qml.init.basic_entangler_layers_normal(n_variational_layers, n_wires)
variational_classifier_linear(initial_params)
# should give 0.48

tensor(0.48339844, requires_grad=True)

In [10]:
desired_output = 1

In [11]:
opt = qml.NesterovMomentumOptimizer(0.1)

params = initial_params
n_iterations = 1
for j in range(n_iterations):
    params, prev_cost = opt.step_and_cost(lambda p: squareloss_cost(p, desired_output), params)

variational_classifier_linear(params)

# should give 0.49

tensor(0.48925781, requires_grad=True)

In [12]:
# multiple criteria can be applied
AwsDevice.get_devices(statuses=['ONLINE'])

[Device('name': Advantage_system1.1, 'arn': arn:aws:braket:::device/qpu/d-wave/Advantage_system1),
 Device('name': Aspen-9, 'arn': arn:aws:braket:::device/qpu/rigetti/Aspen-9),
 Device('name': DW_2000Q_6, 'arn': arn:aws:braket:::device/qpu/d-wave/DW_2000Q_6),
 Device('name': IonQ Device, 'arn': arn:aws:braket:::device/qpu/ionq/ionQdevice),
 Device('name': SV1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/sv1),
 Device('name': TN1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/tn1)]

In [13]:
#device_arn = "arn:aws:braket:::device/qpu/rigetti/Aspen-9"
#s3_folder = ("amazon-braket-amazon-braket-qhack-reply-2021", "rigetti-test_2")
#rigetti = qml.device(
#    "braket.aws.qubit",
#    device_arn=device_arn,
#    wires=n_wires,
#    s3_destination_folder=s3_folder,
#    parallel=True,
#    shots=1024,
#)
#rigetti

<BraketAwsQubitDevice device (wires=2, shots=1024) at 0x7f99b3ac8110>

In [14]:
qpu_circuit = qml.QNode(circuit, rigetti)

In [15]:
def variational_classifier_linear_qpu(params):
    meas = qpu_circuit(params)
    rescaled = (meas + 1) / 2
    return rescaled

In [16]:
def squareloss_cost_qpu(params, label: int) -> float:
    prediction = variational_classifier_linear_qpu(params)
    return squareloss([prediction], [label])

In [17]:
#n_variational_layers = 1
#np.random.seed(2165)
#initial_params = qml.init.basic_entangler_layers_normal(n_variational_layers, n_wires)
#variational_classifier_linear_qpu(initial_params)
## should give 0.48

tensor(0.54199219, requires_grad=True)

In [18]:
# THE CELL ABOVE RUNS IN 1 TASK

In [19]:
desired_output = 1

In [20]:
#opt = qml.NesterovMomentumOptimizer(0.2)
#
#params = initial_params
#n_iterations = 1
#for j in range(n_iterations):
#    params, prev_cost = opt.step_and_cost(lambda p: squareloss_cost_qpu(p, desired_output), params)
#
#variational_classifier_linear_qpu(params)
#
## should give 0.49

tensor(0.52832031, requires_grad=True)

In [ ]:
# THE CELL ABOVE RUNS IN 5 TASKS
# 2 Tasks for each parameter during gradient
# 1 Task for basic circuit at the end

In [21]:
initial_params

tensor([[-0.07350859,  0.14562287]], requires_grad=True)